<a href="https://colab.research.google.com/github/ZhengHuocheng/ZhengHuocheng/blob/main/new_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.7/250.7 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 97.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 99.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 67.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 75.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.5/206.5 kB 28

In [1]:
import json
with open('train-v2.0.json', 'r') as f:
    train_data = json.load(f)
train_data = [item for topic in train_data['data'] for item in topic['paragraphs'] ]

In [2]:
# train_data = train_data[0:10000]
train_data = train_data
split_index = int(len(train_data)*0.7)

In [3]:
train = train_data[0:split_index]
test = train_data[split_index:]
print(len(train))
print(len(test))

13324
5711


In [4]:
import logging

from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs

In [5]:
model_type="bert"
model_name= "bert-base-cased-final"
if model_type == "bert":
    model_name = "bert-base-cased"

elif model_type == "roberta":
    model_name = "roberta-base"

elif model_type == "distilbert":
    model_name = "distilbert-base-cased"

elif model_type == "distilroberta":
    model_type = "roberta"
    model_name = "distilroberta-base"

elif model_type == "electra-base":
    model_type = "electra"
    model_name = "google/electra-base-discriminator"

elif model_type == "electra-small":
    model_type = "electra"
    model_name = "google/electra-small-discriminator"

elif model_type == "xlnet":
    model_name = "xlnet-base-cased"

In [ ]:
!pip install wandb 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
### Advanced Methodology
train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "use_cached_eval_features": True,
    "output_dir": f"outputs/{model_type}",
    "best_model_dir": f"outputs/{model_type}/best_model",
    "evaluate_during_training": False,
    "max_seq_length": 128,
    "num_train_epochs": 5,
    "evaluate_during_training_steps": 1000,
    "wandb_project": "Question Answer Application",
    # "wandb_kwargs": {
    #     "name": model_name,
    #     "config": {
    #         "monitor": "eval_loss",
    #     }
    # },
    "wandb_kwargs": {"name": model_name},
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "n_best_size":3,
    # "use_early_stopping": True,
    # "early_stopping_metric": "mcc",
    # "n_gpu": 2,
    # "manual_seed": 4,
    # "use_multiprocessing": False,
    "train_batch_size": 128,
    "eval_batch_size": 64,
    # "config": {
    #     "output_hidden_states": True
    # }
}

In [7]:
model = QuestionAnsweringModel(
    model_type,model_name, args=train_args
)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForQuestionAnswering: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and a

In [8]:
model.train_model(train)

add example index and unique id: 100%|██████████| 89556/89556 [00:00<00:00, 932704.01it/s]


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


...

Running Epoch 1 of 5:   0%|          | 0/707 [00:00<?, ?it/s]

Running Epoch 2 of 5:   0%|          | 0/707 [00:00<?, ?it/s]

Running Epoch 3 of 5:   0%|          | 0/707 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/707 [00:00<?, ?it/s]

(3535, 0.7107627922397691)

In [ ]:
result, texts = model.eval_model(test)

add example index and unique id: 100%|██████████| 22149/22149 [00:00<00:00, 589715.42it/s]


Running Evaluation:   0%|          | 0/350 [00:00<?, ?it/s]

In [ ]:
model.save_model("path/to/save/directory")

In [ ]:
from google.colab import drive

# 挂载Colab云盘
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
model.save_model("/content/drive/MyDrive/model_directory")

In [ ]:
result

{'correct': 10921,
 'similar': 9902,
 'incorrect': 1326,
 'eval_loss': -7.495345982142857}

{'correct': 10921,
 'similar': 9902,
 'incorrect': 1326,
 'eval_loss': -7.495345982142857}

In [ ]:

to_predict = [
    {
        "context": "Jijun is the most handsome man.",
        "qas": [
            {
                "question": "How to describe Jijun?",
                "id": "0",
            }
        ],
    }
]

In [ ]:

answers, probabilities = model.predict(to_predict)

print(answers)
print(probabilities)     

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 3158.36it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[{'id': '0', 'answer': ['the most handsome man', 'the most handsome', 'most handsome man']}]
[{'id': '0', 'probability': [0.4894678716556123, 0.26199327219653284, 0.2485351911490471]}]


In [ ]:
import shutil
import zipfile
from google.colab import drive

# 要下载的文件夹路径
folder_path = "/content/outputs/bert"

# 压缩文件保存路径
zip_file_path = "/content/drive/MyDrive/QA"

# 将ZIP文件保存到云盘
shutil.move(folder_path, zip_file_path)


'/content/drive/MyDrive/QA'